## FINAL PROJECT. Решение комплексной бизнес-задачи. Создание рекомендательной системы курсов для онлайн-школы MasterMind.

####  Цель : подготовить основу рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы и тем самым повышать средний чек.

Формализованные задачи: 
- Подготовить и проанализировать данные с помощью SQL
- Обработать данные средствами Python
- Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями
- Протестировать фичу

Итоговая таблица должна состоять из трёх столбцов:
- 1.Курс, к которому идёт рекомендация.
- 2.Курс для рекомендации № 1 (самый популярный).
- 3.Курс для рекомендации № 2 (второй по популярности).


### Обработка данных

In [1]:
import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np

In [2]:
# Создадим функцию, которая выполнит SQL-запрос:
# Получающий данные по продажам курсов в разрезе пользователей 
def getCourseUsers():
    query = '''WITH relevant_clients as (
    SELECT DISTINCT user_id,
      COUNT(DISTINCT ci.resource_id) quantity
    FROM final.carts c
    JOIN final.cart_items ci ON c.id = ci.cart_id
    WHERE state = 'successful' AND resource_type = 'Course'
    GROUP BY 1
    HAVING COUNT(DISTINCT ci.resource_id) > 1)

    SELECT DISTINCT c.user_id,
       ci.resource_id
    FROM relevant_clients rc 
    JOIN final.carts c ON rc.user_id = c.user_id
    JOIN final.cart_items ci ON c.id = ci.cart_id
    WHERE state = 'successful' AND resource_type = 'Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
# Выполним функцию запроса и запишем полученные данные в датафрейм
client_course_df = pd.DataFrame(getCourseUsers())


In [3]:
client_course_df.head()

,user_id,resource_id
0,909757,356
1,583850,515
2,1559882,566
3,970967,679
4,1640443,566


In [4]:
# подсчитаем кол-во уникальнх пользователей
client_course_df['user_id'].nunique()

12656

Датафрейм содержит данные о 12656 уникальных пользователях, купивших более 1 курса.

In [5]:
# Создадим датафрейм со списком всех купленных курсов для каждого пользователя
course_pivot = client_course_df.groupby('user_id')['resource_id'].apply(lambda x: x.tolist()).reset_index()
# Отсортируем получившиеся списки
course_pivot['resource_id'] = course_pivot['resource_id'].apply(lambda x: sorted(x))

In [6]:
# Разобъем все покупки пользователей по парам со всеми возможными комбинациями
# Используем функцию combinations, чтобы избежать зеркальных пар
from itertools import combinations
course_pair_list = []
for i in course_pivot['resource_id']:
    for pair_id in combinations(i,2):
        course_pair_list.append(pair_id)
# Ранжируем полученные пары по частности совместных продаж посредством counter и запишем все в словарь отсортированный по убыванию
from collections import Counter
range_pairs = dict(Counter(course_pair_list).most_common())

### Составление таблицы рекомендаций.

Перед тем как составить рекомендации, необходимо определить минимальную границу по частоте совместных покупок курсов, т.е. отсечь случайные. Для этого создадим временный датафрейм из значений словаря 'range_pairs' для оценки статистики выборки.

In [8]:
df_dict = pd.DataFrame(data= range_pairs.values(),columns=['sales_quantity'])
df_dict.describe()

,sales_quantity
count,3989.000000
mean,10.031838
std,26.355998
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,797.000000


Описательная статистика показывает большой разброс значений со стандартным отклонением в 26,35, значит будем использовать более широкий доверительный интервал равный 95%. 

In [9]:
# Найдем нижнюю границу для случайных значений выборки
lower_border = int(df_dict.quantile(q=0.95))
round(lower_border)

37

In [10]:
# Создадим функцию, которая будет на вход принимать id курса и подбирать для него 2 рекомендации из словаря range_pairs 
def recommendation(id):
    recommend_list = []
    for i in range_pairs.keys():
# Если значение id найдено в паре и ранг пары больше установленной границы частности, добавлять соответствующее ему второе значение в список
        if i[0] == id and range_pairs[i] > lower_border:
            recommend_list.append(i[1])
        elif i[1] == id  and range_pairs[i] > lower_border:
            recommend_list.append(i[0])
# Оставим только 2 самых часто встречающихся значения
    return recommend_list[:2]

In [11]:
# Создадим список уникальных id курсов
uniq_id_list = list(client_course_df.resource_id.unique())

# Создадим датафрейм-основу рекоммендательной таблицы
course_recommend_df = pd.DataFrame(columns=['Course','1_st_recommendation','2_nd_recommendation'])

# Запустим цикл по списку с уникальными id, которым будут через функцию recommendation подбираться рекомендации
# Из-за нижней границы ранга не всем курсам подберется по две рекоммендации, недостающие значения заместим на nan
for i in uniq_id_list:
    if len(recommendation(i)) == 2:
        course_recommend_df.loc[i] = [i, recommendation(i)[0], recommendation(i)[1]]
    elif len(recommendation(i)) == 1:
        course_recommend_df.loc[i] = [i, recommendation(i)[0], np.nan]
    else:
        course_recommend_df.loc[i] = [i, np.nan, np.nan]
course_recommend_df = course_recommend_df.sort_values(['Course']).reset_index(drop=True)

In [12]:
# Посмотрим полученный датафрейм
course_recommend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Course               126 non-null    float64
 1   1_st_recommendation  63 non-null     float64
 2   2_nd_recommendation  51 non-null     float64
dtypes: float64(3)
memory usage: 3.1 KB


Замену позиций с NaN будем осуществлять следующим образом: 
- если подобралась только 1-ая рекомендация, то 2-ая рекомендация будет из списка рекомендаций, который подобрался для 1-ой, чтобы сохранить алгоритм совместных частых покупок
- если же к курсу не подобралось ни одной рекомендации, то будем заменять на наши самые продаваемые курсы, кроме 1-ого самого  популярного, так как скорее всего о платформе узнали как раз из-за него, а мы не хотим быть платформой одного курса

In [13]:
# Найдем топ-2 самых продаваемых курса, помимо первого 
# Для этого воспользуемся изначальным выгруженным датафреймом и подсчитаем частоту покупок каждого курса
top_2 = list(client_course_df.resource_id.value_counts().index)[1:3]
top_2

[566, 515]

In [14]:
# Произведем замену значений NaN по вышеописанному алгоритму
# Заменим значения для курсов, не получивших ни одной рекомендации
course_recommend_df.loc[((course_recommend_df['1_st_recommendation'].isna()) & (course_recommend_df['2_nd_recommendation'].isna())), \
    ('1_st_recommendation','2_nd_recommendation')] = 566,515
# Заменим значения для курсов с одной рекоммендацией
for i in course_recommend_df.index:
# Если значение nan, то заменить  на рекомедацию для 1-ого подобранного курса    
    if np.isnan(course_recommend_df.loc[i,'2_nd_recommendation']):
        course_recommend_df.loc[i,'2_nd_recommendation'] = recommendation(course_recommend_df.loc[i,'1_st_recommendation'])[0]

In [15]:
# Проверим целостность заполнения таблицы
course_recommend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Course               126 non-null    float64
 1   1_st_recommendation  126 non-null    float64
 2   2_nd_recommendation  126 non-null    float64
dtypes: float64(3)
memory usage: 3.1 KB


In [17]:
# Импортируем датафрейм в файл 
course_recommend_df.to_csv('course_recommend.csv', index=False, sep=';')

## Тестирование гипотезы

Каждый из вариантов A/B-теста достиг необходимого размера выборки, и теперь необходимо сделать вывод, можно ли считать реализацию рекомендательной системы успешной, и принять решение о полезности её внедрения.

Численное выражение полученных результатов таково:
- В контрольной группе оказалось 8732 клиента, оформивших заказ, из них 293 купили больше одного курса.
- В тестовой — 8847 клиентов, из них 347 купили больше одного курса.

In [23]:
a_group = 8732
b_group = 8847
relation = a_group/b_group
round(relation,2)

0.99

Продолжаем тест, так как разница не превышает 1%.

In [26]:
from scipy import stats as st
import math as mth

Гипотезы:
- Н0: Конверсии обеих групп равны
- Н1: Конверсии группы В > A 

In [29]:
# Задаём желаемый уровень статистической значимости: 
a = .05
x_A = 293
x_B = 347
n_A = a_group
n_B = b_group
p_A = x_A / n_A
p_B = x_B / n_B
p = (x_A + x_B) / (n_A + n_B)
# Посчитаем разницу в пропорциях: 
diff = p_A - p_B
# Рассчитаем Z-статистику:
z = diff / mth.sqrt(p * (1 - p) * (1/n_A + 1/n_B))
# Зададим нормальное стандартное распределение со средним, равным нулю, и стандартным отклонением, равным единице:
distr = st.norm(0, 1)
# Рассчитаем p-value, примепним односторонний тест :
z_p_val = (1 - distr.cdf(abs(z)))
print('P-value равен', z_p_val)

P-value равен 0.02243159896524971


P-value меньше принятого уровня значимости, следовательно, нулевая гипотеза не оправдалась.

Конверсия группы b (3.92%) была на 16.6% выше чем конверсия группы a (3.36%). С вероятностью 95% можно быть уверенным, что полученнй результат конверсии был достигнут благодаря введению новой фичи, а не случайностью.

## Вывод

В таблице представлено 126 курсов (всего у нас есть данные о 127 курсах, но один из курсов не попал в выборку, т.к. нет данных по его покупке) с рекомендациями, которые опираются на частоту совместных покупок пар курсов. 

Нижняя граница частоты совместных покупок с  95% доверительным интервалом  должна быть больше 37 раз, все пары курсы купленные реже нижней границы приняты случайными и не использовались.

Таким образом используя данный алгоритм удалось заполнить 50% таблицы, еще 50% таблицы заполнили рекомендуя курсы, которые покупали вместе с одним из уже предложенных, а для курсов, которым не нашлось ни одной рекомендации изначально, предложили наши топ-2 и топ-3 самых продаваемых курса ( так как топ-1 итак достаточно популярный и не нуждается в рекламе).

Тестирование показало: вариант с рекомендациями показал статистически значимо лучший результат, нужно реализовывать его для всех клиентов. 